In [2]:
!pip install jaconv

  Preparing metadata (setup.py) ... done
  Created wheel for jaconv: filename=jaconv-0.4.0-py3-none-any.whl size=18228 sha256=e885408ad4c75f894b7b2676341a448954068cb82069da262e60d52bd6179c93
  Stored in directory: /root/.cache/pip/wheels/c9/97/53/1f827ebf916b899520ce663227497206525c432be72cc29265
Successfully built jaconv


In [3]:
# ライブラリのインポート
import pandas as pd
from google.colab import files
import io
import jaconv
import re

In [4]:
# 料理データセットの読み込み
uploaded = files.upload()  # ファイル選択ダイアログを出す
file_name = list(uploaded.keys())[0]
xls = pd.ExcelFile(io.BytesIO(uploaded[file_name]))  # Excelファイルのシート名を確認
print(xls.sheet_names)  # Excelのシート名の確認

df_recipes_ori = pd.read_excel(xls, sheet_name='recipes')
df_prebiotics_ori = pd.read_excel(xls, sheet_name='prebiotics')
df_probiotics_ori = pd.read_excel(xls, sheet_name='probiotics')

Saving 料理名と食材・栄養素のデータセット.xlsx to 料理名と食材・栄養素のデータセット.xlsx
['recipes', 'prebiotics', 'probiotics']


In [5]:
df_recipes = df_recipes_ori.copy()
df_prebiotics = df_prebiotics_ori.copy()
df_probiotics = df_probiotics_ori.copy()

In [6]:
# 列をすべて表示
pd.set_option('display.max_columns', None)

# 行をすべて表示
pd.set_option('display.max_rows', None)
df_recipes.head()

,料理の種類,料理名,エネルギー[キロカロリー],たんぱく質[グラム],脂質[グラム],炭水化物[グラム],食塩相当量[グラム],野菜量[グラム],材料1＿材料名,材料1＿重量[グラム],材料2＿材料名,材料2＿重量[グラム],材料3＿材料名,材料3＿重量[グラム],材料4＿材料名,材料4＿重量[グラム],材料5＿材料名,材料5＿重量[グラム],材料6＿材料名,材料6＿重量[グラム],材料7＿材料名,材料7＿重量[グラム],材料8＿材料名,材料8＿重量[グラム],材料9＿材料名,材料9＿重量[グラム],材料10＿材料名,材料10＿重量[グラム],材料11＿材料名,材料11＿重量[グラム],材料12＿材料名,材料12＿重量[グラム],材料13＿材料名,材料13＿重量[グラム],材料14＿材料名,材料14＿重量[グラム],材料15＿材料名,材料15＿重量[グラム],材料16＿材料名,材料16＿重量[グラム],材料17＿材料名,材料17＿重量[グラム],材料18＿材料名,材料18＿重量[グラム],材料19＿材料名,材料19＿重量[グラム],材料20＿材料名,材料20＿重量[グラム]
0,ごはん,ごはん（中茶碗1杯）,252,3.8,0.5,55.7,0.0,0.0,めし・精白米（水稲）,150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ごはん,麦ごはん,248,4.3,0.7,54.0,0.0,0.0,米・精白米（水稲）,60.0,大麦・押麦,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ごはん,きのこたっぷり混ぜご飯,307,6.6,2.4,61.4,1.0,23.0,米・精白米（水稲）,70.0,えのきたけ＿生,7.0,しめじ・ぶなしめじ＿生,7.0,まいたけ＿生,7.0,油揚げ,5.0,かつお・昆布だし,90.0,こいくちしょうゆ,6.0,みりん風調味料,9.0,清酒・上撰,7.5,糸みつば・葉＿生,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ごはん,五目炊き込みご飯,311,11.4,2.7,57.9,1.1,35.0,米・精白米（水稲）,66.0,油揚げ,5.0,にんじん・根、皮つき＿生,7.0,板こんにゃく（精粉こんにゃく）,15.0,ごぼう・根＿生,20.0,生しいたけ＿生,5.0,若鶏・むね、皮なし＿生,25.0,こいくちしょうゆ,3.0,みりん風調味料,3.0,昆布だし,50.0,食塩,0.5,清酒・上撰,1.2,さやいんげん・若ざや＿生,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ごはん,赤飯,289,6.1,1.1,63.8,0.5,0.0,赤飯（もち米製品）,150.0,ごま＿いり,1.0,食塩,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# プレバイオティクスのデータの確認
df_prebiotics.head()

,野菜類,豆類,フルーツ,海藻類,きのこ類,その他
0,アスパラガス,大豆,バナナ,わかめ,しいたけ,はちみつ
1,ブロッコリー,レンズ豆,りんご,昆布,えのきたけ,NaN
2,キャベツ,ひよこ豆,ベリー,ひじき,まいたけ,NaN
3,玉ねぎ,インゲン豆,キウイ,のり,えのき,NaN
4,ごぼう,NaN,NaN,NaN,NaN,NaN


# 料理データセットの前処理

In [8]:
# 材料の列の欠損値を" - "で穴埋めする
df_recipes.iloc[:,8::] = df_recipes.iloc[:,8::].fillna('-');

/tmp/ipython-input-3194469072.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0          -
1       10.0
2        7.0
3        5.0
4        1.0
5       3.75
6       3.75
7       20.0
8       75.0
9       80.0
10      70.0
11      40.0
12      20.0
13      60.0
14      30.0
15      25.0
16       7.0
17       3.0
18       6.0
19       6.0
20      20.0
21      40.0
22         -
23       6.0
24      21.0
25      22.0
26       6.0
27      25.0
28         -
29         -
30         -
31         -
32     300.0
33     300.0
34     300.0
35     300.0
36     300.0
37     300.0
38     300.0
39     300.0
40     300.0
41      10.0
42     300.0
43     300.0
44     300.0
45      30.0
46      20.0
47      70.0
48      30.0
49      40.0
50       6.0
51       9.0
52       4.0
53      10.0
54       7.5
55       5.0
56       6.7
57      40.0
58      30.0
59      25.0
60      50.0
61      15.0
62       1.0
63       0.5
64       0.4
6

In [9]:
df_recipes.head()

,料理の種類,料理名,エネルギー[キロカロリー],たんぱく質[グラム],脂質[グラム],炭水化物[グラム],食塩相当量[グラム],野菜量[グラム],材料1＿材料名,材料1＿重量[グラム],材料2＿材料名,材料2＿重量[グラム],材料3＿材料名,材料3＿重量[グラム],材料4＿材料名,材料4＿重量[グラム],材料5＿材料名,材料5＿重量[グラム],材料6＿材料名,材料6＿重量[グラム],材料7＿材料名,材料7＿重量[グラム],材料8＿材料名,材料8＿重量[グラム],材料9＿材料名,材料9＿重量[グラム],材料10＿材料名,材料10＿重量[グラム],材料11＿材料名,材料11＿重量[グラム],材料12＿材料名,材料12＿重量[グラム],材料13＿材料名,材料13＿重量[グラム],材料14＿材料名,材料14＿重量[グラム],材料15＿材料名,材料15＿重量[グラム],材料16＿材料名,材料16＿重量[グラム],材料17＿材料名,材料17＿重量[グラム],材料18＿材料名,材料18＿重量[グラム],材料19＿材料名,材料19＿重量[グラム],材料20＿材料名,材料20＿重量[グラム]
0,ごはん,ごはん（中茶碗1杯）,252,3.8,0.5,55.7,0.0,0.0,めし・精白米（水稲）,150.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,ごはん,麦ごはん,248,4.3,0.7,54.0,0.0,0.0,米・精白米（水稲）,60.0,大麦・押麦,10.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,ごはん,きのこたっぷり混ぜご飯,307,6.6,2.4,61.4,1.0,23.0,米・精白米（水稲）,70.0,えのきたけ＿生,7.0,しめじ・ぶなしめじ＿生,7.0,まいたけ＿生,7.0,油揚げ,5.0,かつお・昆布だし,90.0,こいくちしょうゆ,6.0,みりん風調味料,9.0,清酒・上撰,7.5,糸みつば・葉＿生,2.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
3,ごはん,五目炊き込みご飯,311,11.4,2.7,57.9,1.1,35.0,米・精白米（水稲）,66.0,油揚げ,5.0,にんじん・根、皮つき＿生,7.0,板こんにゃく（精粉こんにゃく）,15.0,ごぼう・根＿生,20.0,生しいたけ＿生,5.0,若鶏・むね、皮なし＿生,25.0,こいくちしょうゆ,3.0,みりん風調味料,3.0,昆布だし,50.0,食塩,0.5,清酒・上撰,1.2,さやいんげん・若ざや＿生,3.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,ごはん,赤飯,289,6.1,1.1,63.8,0.5,0.0,赤飯（もち米製品）,150.0,ごま＿いり,1.0,食塩,0.5,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


In [10]:
# 「・」「＿」「（）」以降の単語を削除する関数
def clean_foodname(name):
  """
  IN:鶏卵・全卵＿生
  OUT:鶏卵
  """
  # 「（～）」を削除
  name = re.sub("（[^）]*）", "", name)
  # 「・」「＿」以降も削除
  name = re.sub("[・＿].*", "", name)
  return name

In [11]:
# clean_foodname関数の実行
df_recipes.iloc[:,8::2] = df_recipes.iloc[:,8::2].applymap(clean_foodname)

/tmp/ipython-input-1019100835.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_recipes.iloc[:,8::2] = df_recipes.iloc[:,8::2].applymap(clean_foodname)


In [12]:
# プレバイオティクス一覧を参照し、当てはまればOne-Hot-Encordingする
pre_word = df_prebiotics.stack().tolist() # データフレームの列をリスト化する、「nan」は除外する
# ラベル付け
def check_prebiotics(cell):
  """
  それぞれのレシピにプレバイオティクス一覧の食材が含まれているかどうかを判定する
  OUT:
  含まれている→　1
  含まれていない　→　0
  ※　材料を個別に判断するのではなく、料理全体（1行の中を判断する
  """
  for x in pre_word:
    if x in cell:
      return 1
  return 0

/tmp/ipython-input-2922933683.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_recipes['プレバイオティクス（全体）'] = df_recipes.iloc[:,8::2].applymap(check_prebiotics).max(axis=1)


In [ ]:
# check_prebiotics関数の実行
df_recipes['プレバイオティクス（全体）'] = df_recipes.iloc[:,8::2].applymap(check_prebiotics).max(axis=1)

In [13]:
# プロバイオティクス一覧を参照し、当てはまればOne-Hot-Encordingする
pro_word = df_probiotics.values.ravel().tolist() # データフレームの列をリスト化する

In [14]:
# プロバイオティクスのラベル付け
def check_probiotics(cell):
  """
  それぞれのレシピにプロバイオティクス一覧の食材が含まれているかどうかを判定する
  OUT:
  含まれている→　1
  含まれていない　→　0
  ※　材料を個別に判断するのではなく、料理全体（1行の中）を判断する
  """
  for x in pro_word:
    if x in cell:
      return 1
  return 0

In [15]:
# check_prebiotics関数の実行
# 食材の列の範囲（インデックス8～46までの列）
df_recipes['プロバイオティクス（全体）'] = df_recipes.iloc[:,8:46:2].applymap(check_probiotics).max(axis=1)

/tmp/ipython-input-1699891086.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_recipes['プロバイオティクス（全体）'] = df_recipes.iloc[:,8:46:2].applymap(check_probiotics).max(axis=1)


In [16]:
df_recipes.head()

,料理の種類,料理名,エネルギー[キロカロリー],たんぱく質[グラム],脂質[グラム],炭水化物[グラム],食塩相当量[グラム],野菜量[グラム],材料1＿材料名,材料1＿重量[グラム],材料2＿材料名,材料2＿重量[グラム],材料3＿材料名,材料3＿重量[グラム],材料4＿材料名,材料4＿重量[グラム],材料5＿材料名,材料5＿重量[グラム],材料6＿材料名,材料6＿重量[グラム],材料7＿材料名,材料7＿重量[グラム],材料8＿材料名,材料8＿重量[グラム],材料9＿材料名,材料9＿重量[グラム],材料10＿材料名,材料10＿重量[グラム],材料11＿材料名,材料11＿重量[グラム],材料12＿材料名,材料12＿重量[グラム],材料13＿材料名,材料13＿重量[グラム],材料14＿材料名,材料14＿重量[グラム],材料15＿材料名,材料15＿重量[グラム],材料16＿材料名,材料16＿重量[グラム],材料17＿材料名,材料17＿重量[グラム],材料18＿材料名,材料18＿重量[グラム],材料19＿材料名,材料19＿重量[グラム],材料20＿材料名,材料20＿重量[グラム],プレバイオティクス（全体）,プロバイオティクス（全体）
0,ごはん,ごはん（中茶碗1杯）,252,3.8,0.5,55.7,0.0,0.0,めし,150.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0
1,ごはん,麦ごはん,248,4.3,0.7,54.0,0.0,0.0,米,60.0,大麦,10.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0
2,ごはん,きのこたっぷり混ぜご飯,307,6.6,2.4,61.4,1.0,23.0,米,70.0,えのきたけ,7.0,しめじ,7.0,まいたけ,7.0,油揚げ,5.0,かつお,90.0,こいくちしょうゆ,6.0,みりん風調味料,9.0,清酒,7.5,糸みつば,2.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0
3,ごはん,五目炊き込みご飯,311,11.4,2.7,57.9,1.1,35.0,米,66.0,油揚げ,5.0,にんじん,7.0,板こんにゃく,15.0,ごぼう,20.0,生しいたけ,5.0,若鶏,25.0,こいくちしょうゆ,3.0,みりん風調味料,3.0,昆布だし,50.0,食塩,0.5,清酒,1.2,さやいんげん,3.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0
4,ごはん,赤飯,289,6.1,1.1,63.8,0.5,0.0,赤飯,150.0,ごま,1.0,食塩,0.5,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0


# 腸活スコアの算出

In [17]:
# ①プレバイオティクス・プロバイオティクスの点数
# プレバイオティクス：15 / 30 点　プロバイオティクス：15 / 30点）
prepro_score = 0
df_recipes['']
if df_recipes['プレバイオティクス（全体）'] =

SyntaxError: invalid syntax (ipython-input-2010118650.py, line 5)